In [8]:
import os
import json
import certifi
import pandas as pd

import pymongo

In [9]:
print(f"Running PyMongo Version: {pymongo.__version__}")

Running PyMongo Version: 4.10.1


In [10]:
mongodb_args = {
    "user_name" : "mustafalonandwala750",  
    "password" : "MongodbDS2002",          
    "cluster_name" : "cluster-ds-2002",    
    "cluster_subnet" : "kvu5a",           
    "cluster_location" : "atlas",
    "db_name" : "chinook_dw"              
}

In [11]:
def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client

def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [12]:
client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {
    "customers": 'customer_dimension.json' 
}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         